# MNIST Diffusion 数学推导（高中数学版）

## 核心思想

Diffusion模型的核心思想很简单：**先把图片变成噪声，再学习如何从噪声变回图片**。

---

## 1. 前向过程：加噪声

假设原始图片是 $x_0$（比如一张手写数字图片），我们要逐步往里面加噪声。

### 单步加噪公式

在第 $t$ 步，我们这样加噪声：

$$x_t = \sqrt{\alpha_t} \cdot x_{t-1} + \sqrt{1-\alpha_t} \cdot \epsilon$$

其中：
- $\alpha_t$ 是一个接近1的数（比如0.99），控制保留多少原图
- $\epsilon$ 是标准正态分布噪声（均值0，方差1）
- $\sqrt{\alpha_t}$ 表示保留原图的比例
- $\sqrt{1-\alpha_t}$ 表示加入噪声的比例

**为什么用平方根？** 因为 $(\sqrt{\alpha_t})^2 + (\sqrt{1-\alpha_t})^2 = \alpha_t + (1-\alpha_t) = 1$，这保证了信号的总"能量"不变。

### 多步累积公式

经过 $t$ 步后，可以直接从 $x_0$ 算出 $x_t$：

$$x_t = \sqrt{\bar{\alpha}_t} \cdot x_0 + \sqrt{1-\bar{\alpha}_t} \cdot \epsilon$$

其中 $\bar{\alpha}_t = \alpha_1 \cdot \alpha_2 \cdot \ldots \cdot \alpha_t$（连乘）

**推导**（用 $t=2$ 举例）：

$$x_1 = \sqrt{\alpha_1} \cdot x_0 + \sqrt{1-\alpha_1} \cdot \epsilon_1$$

$$x_2 = \sqrt{\alpha_2} \cdot x_1 + \sqrt{1-\alpha_2} \cdot \epsilon_2$$

代入 $x_1$：

$$x_2 = \sqrt{\alpha_2} \cdot (\sqrt{\alpha_1} \cdot x_0 + \sqrt{1-\alpha_1} \cdot \epsilon_1) + \sqrt{1-\alpha_2} \cdot \epsilon_2$$

$$= \sqrt{\alpha_1 \alpha_2} \cdot x_0 + \sqrt{\alpha_2(1-\alpha_1)} \cdot \epsilon_1 + \sqrt{1-\alpha_2} \cdot \epsilon_2$$

两个噪声项可以合并（因为正态分布的性质：方差可加）：

$$= \sqrt{\bar{\alpha}_2} \cdot x_0 + \sqrt{1-\bar{\alpha}_2} \cdot \epsilon$$

---

## 2. 反向过程：去噪声

现在我们有了噪声图 $x_t$，想恢复原图 $x_0$。

### 神经网络的任务

训练一个神经网络 $\epsilon_\theta(x_t, t)$，输入噪声图 $x_t$ 和时间步 $t$，预测加进去的噪声 $\epsilon$。

### 损失函数

用最简单的均方误差：

$$L = \|\epsilon - \epsilon_\theta(x_t, t)\|^2$$

即：**预测的噪声** 与 **真实噪声** 之间的差距

展开来写：
$$L = \|\epsilon - \epsilon_\theta(\sqrt{\bar{\alpha}_t} \cdot x_0 + \sqrt{1-\bar{\alpha}_t} \cdot \epsilon, t)\|^2$$

---

## 3. 采样（生成图片）

训练好后，从纯噪声 $x_T$ 开始，逐步去噪：

$$x_{t-1} = \frac{1}{\sqrt{\alpha_t}}\left(x_t - \frac{1-\alpha_t}{\sqrt{1-\bar{\alpha}_t}} \cdot \epsilon_\theta(x_t, t)\right) + \sigma_t \cdot z$$

其中 $z$ 是随机噪声，$\sigma_t$ 是噪声系数。

**直觉理解**：
1. $\epsilon_\theta(x_t, t)$ 预测出噪声
2. 从 $x_t$ 中减去噪声的贡献
3. 除以 $\sqrt{\alpha_t}$ 恢复原始信号的尺度
4. 加一点随机噪声保持多样性

---

## 4. 总结

| 步骤 | 公式 | 含义 |
|------|------|------|
| 加噪 | $x_t = \sqrt{\bar{\alpha}_t} x_0 + \sqrt{1-\bar{\alpha}_t} \epsilon$ | 原图+噪声的加权混合 |
| 训练 | $L = \|\epsilon - \epsilon_\theta(x_t, t)\|^2$ | 让网络学会预测噪声 |
| 去噪 | 从 $x_T$ 逐步减去预测的噪声 | 从噪声恢复图片 |

**就这么简单！** 整个过程只用到了：
- 加权平均
- 均方误差
- 正态分布的基本性质

# Stable Diffusion 架构全景图

## 整体流程图

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                         Stable Diffusion 生成流程                            │
└─────────────────────────────────────────────────────────────────────────────┘

                    ┌──────────────────┐
                    │   "a cute cat"   │  ← 用户输入的文本提示词
                    └────────┬─────────┘
                             │
                             ▼
              ┌──────────────────────────────┐
              │     CLIP Text Encoder        │  ← 把文字变成向量
              │  (Transformer, 77 tokens)    │
              └──────────────┬───────────────┘
                             │
                             ▼
                 ┌───────────────────────┐
                 │   Text Embeddings     │  ← 文本的数学表示
                 │   (77 × 768 向量)     │     理解了"猫"的语义
                 └───────────┬───────────┘
                             │
        ┌────────────────────┼────────────────────┐
        │                    │                    │
        │                    ▼                    │
        │    ┌───────────────────────────────┐    │
        │    │         UNET (核心!)          │    │
        │    │  ┌─────────────────────────┐  │    │
        │    │  │ Cross-Attention Layers  │◄─┼────┘  文本条件注入
        │    │  └─────────────────────────┘  │
        │    │  ┌─────────────────────────┐  │
        │    │  │   ResNet Blocks         │  │
        │    │  └─────────────────────────┘  │
        │    │  ┌─────────────────────────┐  │
        │    │  │   Self-Attention        │  │
        │    │  └─────────────────────────┘  │
        │    └───────────────┬───────────────┘
        │                    │
        │                    │ 预测噪声 ε
        │                    ▼
        │    ┌───────────────────────────────┐
        │    │        Scheduler              │  ← 控制去噪步骤
        │    │   (DDPM / DDIM / Euler...)    │    决定每步减多少噪声
        │    └───────────────┬───────────────┘
        │                    │
        │                    ▼
        │         ┌─────────────────┐
        └────────►│  Latent Image   │  ← 潜空间中的图像 (64×64×4)
    循环迭代      │   x_{t-1}       │    比原图小 8 倍！
   (50步左右)    └────────┬────────┘
                          │
                          │ 去噪完成
                          ▼
              ┌──────────────────────────────┐
              │       VAE Decoder            │  ← 放大 + 还原细节
              │   (潜空间 → 像素空间)         │
              └──────────────┬───────────────┘
                             │
                             ▼
                 ┌───────────────────────┐
                 │    Final Image        │  ← 512×512 RGB图像
                 │   🐱 一只可爱的猫      │
                 └───────────────────────┘


┌─────────────────────────────────────────────────────────────────────────────┐
│  训练时还需要 VAE Encoder: 真实图片 → 潜空间，用于生成训练数据              │
└─────────────────────────────────────────────────────────────────────────────┘
```

---

## 各组件分工说明

| 组件 | 输入 | 输出 | 核心作用 |
|------|------|------|----------|
| **CLIP Text Encoder** | 文本字符串 | 77×768 向量 | 理解文本语义，把"猫"变成数学向量 |
| **VAE Encoder** | 512×512 图像 | 64×64×4 潜向量 | 压缩图像（训练时用） |
| **VAE Decoder** | 64×64×4 潜向量 | 512×512 图像 | 还原图像细节 |
| **UNET** | 噪声潜向量 + 文本向量 + 时间步 | 预测的噪声 | **核心！** 学习去噪，生成图像内容 |
| **Scheduler** | 当前噪声 + 预测噪声 | 下一步的噪声 | 控制去噪节奏和策略 |

---

## 为什么在潜空间（Latent Space）工作？

这是 Stable Diffusion 的关键创新：

```
原始图像空间:  512 × 512 × 3  = 786,432 个数值  😰 太大了！
潜空间:        64 ×  64 × 4  =  16,384 个数值  😊 小 48 倍！
```

- **省内存**：UNET 处理的数据量大大减少
- **省计算**：去噪迭代 50 次，每次都省很多
- **保质量**：VAE 学会了保留图像的关键信息

---

## Cross-Attention：文字如何控制图像？

UNET 中的 Cross-Attention 是"魔法"发生的地方：

```
Q (Query)  ← 来自图像特征：「图像的某个位置想要什么？」
K (Key)    ← 来自文本向量：「文本中有哪些概念？」  
V (Value)  ← 来自文本向量：「这些概念的具体含义是什么？」

Attention = softmax(Q × K^T) × V

结果：图像的每个位置都"查询"文本，找到相关的语义信息
      比如：生成眼睛的位置会关注"cat"这个词
```

---

## 一句话总结

> **CLIP 理解文字 → UNET 在潜空间按文字指导去噪 → VAE 把结果放大成真实图片**